In [ ]:
import pandas as pd
import numpy as np
import os

# Load both spring and fall CSVs
file_path_fall = os.path.join('Source', 'DCMSBL18+OwlsStatsFall2024.csv')
file_path_spring = os.path.join('Source', 'DCMSBL18+OwlsStatsSpring2024.csv')
fall_stats_df = pd.read_csv(file_path_fall, header=1)
spring_stats_df = pd.read_csv(file_path_spring, header=1)

fall_stats_df = fall_stats_df[:19]
spring_stats_df = spring_stats_df[:21]

# Ensure columns are consistent
fall_stats_df.columns = fall_stats_df.columns.map(lambda x: str(x).strip())
spring_stats_df.columns = spring_stats_df.columns.map(lambda x: str(x).strip())

# Ensure player names match between DataFrames for proper alignment
fall_stats_df['Full_Name'] = fall_stats_df['First'] + ' ' + fall_stats_df['Last']
spring_stats_df['Full_Name'] = spring_stats_df['First'] + ' ' + spring_stats_df['Last']

# List of columns that should be numeric
numeric_columns = [
    'QAB%', 'C%', 'BA/RISP', 'AVG', 'OBP', 'OPS', 'SLG',
    # Add other numeric columns if necessary
]

# Convert columns to numeric in both DataFrames
for col in numeric_columns:
    if col in fall_stats_df.columns:
        fall_stats_df[col] = pd.to_numeric(fall_stats_df[col], errors='coerce')
    if col in spring_stats_df.columns:
        spring_stats_df[col] = pd.to_numeric(spring_stats_df[col], errors='coerce')

# Merge the dataframes by averaging numeric columns
merged_stats_df = pd.merge(
    fall_stats_df,
    spring_stats_df,
    on=['Full_Name', 'First', 'Last'],
    how='outer',
    suffixes=('_fall', '_spring')
)

# Identify all columns with '_fall' and '_spring' suffixes
fall_columns = [col for col in merged_stats_df.columns if col.endswith('_fall')]
spring_columns = [col for col in merged_stats_df.columns if col.endswith('_spring')]

# Remove suffixes to get base column names
fall_base_cols = [col[:-5] for col in fall_columns]  # Remove '_fall'
spring_base_cols = [col[:-7] for col in spring_columns]  # Remove '_spring'

# Get the set of all base columns
all_base_cols = set(fall_base_cols).union(set(spring_base_cols))

# For each base column, handle averaging and missing data
for col in all_base_cols:
    fall_col = f'{col}_fall'
    spring_col = f'{col}_spring'

    # Check if columns exist and convert to numeric
    cols_to_average = []
    if fall_col in merged_stats_df.columns:
        merged_stats_df[fall_col] = pd.to_numeric(merged_stats_df[fall_col], errors='coerce')
        cols_to_average.append(merged_stats_df[fall_col])
    else:
        merged_stats_df[fall_col] = np.nan  # Create column with NaN if it doesn't exist

    if spring_col in merged_stats_df.columns:
        merged_stats_df[spring_col] = pd.to_numeric(merged_stats_df[spring_col], errors='coerce')
        cols_to_average.append(merged_stats_df[spring_col])
    else:
        merged_stats_df[spring_col] = np.nan  # Create column with NaN if it doesn't exist

    # Calculate the mean across the available columns
    if cols_to_average:
        merged_stats_df[col] = pd.concat(cols_to_average, axis=1).mean(axis=1)
    else:
        merged_stats_df[col] = np.nan  # If no data, set as NaN

# Convert necessary columns to numeric again (after averaging)
for col in numeric_columns:
    if col in merged_stats_df.columns:
        merged_stats_df[col] = pd.to_numeric(merged_stats_df[col], errors='coerce')

# Read Baserunning.xlsx and merge
baserunning_df = pd.read_excel(os.path.join('Source', 'Baserunning.xlsx'))
baserunning_df.columns = baserunning_df.columns.map(lambda x: str(x).strip())
baserunning_df['First'] = baserunning_df['First'].astype(str).str.strip()
baserunning_df['Last'] = baserunning_df['Last'].astype(str).str.strip()

# Merge baserunning data
merged_stats_df = pd.merge(
    merged_stats_df,
    baserunning_df[['First', 'Last', 'Baserunning']],
    on=['First', 'Last'],
    how='left'
)

# Fill missing Baserunning values with 0
merged_stats_df['Baserunning'] = merged_stats_df['Baserunning'].fillna(0)

# Convert 'First' and 'Last' columns to strings and fill NaNs with empty strings
merged_stats_df['First'] = merged_stats_df['First'].astype(str).fillna('')
merged_stats_df['Last'] = merged_stats_df['Last'].astype(str).fillna('')

# Store original 'First' and 'Last' names for output
merged_stats_df['First_Original'] = merged_stats_df['First']
merged_stats_df['Last_Original'] = merged_stats_df['Last']

# Convert 'First' and 'Last' columns to lowercase for matching
merged_stats_df['First'] = merged_stats_df['First'].str.lower().str.strip()
merged_stats_df['Last'] = merged_stats_df['Last'].str.lower().str.strip()

# Add 'Baserunning' to numeric_columns and ranking_columns
numeric_columns.append('Baserunning')
ranking_columns = ['AVG', 'OBP', 'OPS', 'SLG', 'QAB%', 'C%', 'BA/RISP', 'Baserunning']
# Note: Removed 'QAB%' since we have 8 metrics now

# Rank the players for each metric and assign weights based on lineup groups
for col in ranking_columns:
    if col in merged_stats_df.columns:
        merged_stats_df[f'{col}_rank'] = merged_stats_df[col].rank(ascending=False)

# Define weights for each group
group_weights = {
    'Leadoff': {
        'AVG_rank': 0.20,
        'OBP_rank': 0.35,
        'SLG_rank': 0.00,
        'OPS_rank': 0.10,
        'C%_rank': 0.00,
        'Baserunning_rank': 0.35,
        'BA/RISP_rank': 0.00,
        'QAB%_rank': 0.0
    },
    '2 spot': {
        'AVG_rank': 0.14,
        'OBP_rank': 0.22,
        'SLG_rank': 0.10,
        'OPS_rank': 0.12,
        'C%_rank': 0.20,
        'Baserunning_rank': 0.14,
        'BA/RISP_rank': 0.00,
        'QAB%_rank': 0.08
    },
    '3-7': {
        'AVG_rank': 0.15,
        'OBP_rank': 0.10,
        'SLG_rank': 0.20,
        'OPS_rank': 0.14,
        'C%_rank': 0.11,
        'Baserunning_rank': 0.1,
        'BA/RISP_rank': 0.12,
        'QAB%_rank': 0.08
    },
    '8+': {
        'AVG_rank': 0.20,
        'OBP_rank': 0.20,
        'SLG_rank': 0.10,
        'OPS_rank': 0.20,
        'C%_rank': 0.10,
        'Baserunning_rank': 0.00,
        'BA/RISP_rank': 0.10,
        'QAB%_rank': 0.10
    }
}

# Function to compute weighted rank
def compute_weighted_rank(df, weights):
    weighted_rank = pd.Series(0, index=df.index)
    for key, weight in weights.items():
        if key in df.columns:
            weighted_rank += df[key] * weight
        else:
            # Column does not exist; handle as needed
            pass
    return weighted_rank

# Generate weighted lineup for each group and pop players once assigned
final_lineup = []

# Make a copy of merged_stats_df to avoid modifying the original during iteration
lineup_df = merged_stats_df.copy()

# Assign the Leadoff player
leadoff_weights = group_weights['Leadoff']
lineup_df['Weighted_rank'] = compute_weighted_rank(lineup_df, leadoff_weights)
leadoff_player = lineup_df.sort_values(by='Weighted_rank').head(1)
final_lineup.append(leadoff_player[['First_Original', 'Last_Original', 'Weighted_rank']])
lineup_df = lineup_df.drop(leadoff_player.index)

# Assign the 2 spot player
spot_2_weights = group_weights['2 spot']
lineup_df['Weighted_rank'] = compute_weighted_rank(lineup_df, spot_2_weights)
spot_2_player = lineup_df.sort_values(by='Weighted_rank').head(1)
final_lineup.append(spot_2_player[['First_Original', 'Last_Original', 'Weighted_rank']])
lineup_df = lineup_df.drop(spot_2_player.index)

# Assign the 3-7 spot players (4 players)
spot_3_7_weights = group_weights['3-7']
lineup_df['Weighted_rank'] = compute_weighted_rank(lineup_df, spot_3_7_weights)
spot_3_7_players = lineup_df.sort_values(by='Weighted_rank').head(4)
final_lineup.append(spot_3_7_players[['First_Original', 'Last_Original', 'Weighted_rank']])
lineup_df = lineup_df.drop(spot_3_7_players.index)

# Assign the 8+ spot players (remaining players)
spot_8_weights = group_weights['8+']
lineup_df['Weighted_rank'] = compute_weighted_rank(lineup_df, spot_8_weights)
spot_8_players = lineup_df.sort_values(by='Weighted_rank')
final_lineup.append(spot_8_players[['First_Original', 'Last_Original', 'Weighted_rank']])

# Concatenate all group lineups
final_lineup_df = pd.concat(final_lineup, ignore_index=True)

# Rename the columns back to 'First' and 'Last' for output
final_lineup_df.rename(columns={'First_Original': 'First', 'Last_Original': 'Last'}, inplace=True)